In [1]:
# import os
# import shutil

# # Change directory to 'StyleTTS2'
# try:
#     os.chdir("StyleTTS2")
#     print(f"Changed directory to: {os.getcwd()}")
# except FileNotFoundError:
#     print("Directory 'StyleTTS2' not found.")

Changed directory to: /home/youshanzhang/sahil/projects/sem1/voice_cloning/clone_2/StyleTTS2


In [7]:
import os
import subprocess
import sys
import shutil

# Function to execute commands in a subprocess with error handling
def run_command(command):
    try:
        process = subprocess.run(command, shell=True, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        return process.stdout.decode('utf-8')
    except subprocess.CalledProcessError as e:
        print(f"Error while executing command: {command}\nError message: {e.stderr.decode('utf-8')}")
        return None

# Clone the repository
try:
    print("Cloning StyleTTS2 repository...")
    run_command("git clone https://github.com/yl4579/StyleTTS2.git")
    os.chdir("StyleTTS2")
except Exception as e:
    print(f"Error while cloning repository: {e}")

# Install the required packages
required_packages = [
    "SoundFile",  "munch", "pydub", "pyyaml", "librosa", "nltk",
    "matplotlib", "accelerate", "transformers", "phonemizer", "einops", "einops-exts",
    "tqdm", "typing-extensions", "git+https://github.com/resemble-ai/monotonic_align.git"
]

# for package in required_packages:
#     try:
#         print(f"Installing package: {package}...")
#         run_command(f"{sys.executable} -m pip install {package}")
#     except Exception as e:
#         print(f"Error while installing {package}: {e}")

# Install espeak-ng for Windows
# try:
#     print("Installing espeak-ng...")
#     run_command(f"{sys.executable} -m pip install py-espeak-ng")
# except Exception as e:
#     print(f"Error while installing espeak-ng: {e}")

# Clone the Hugging Face model using Git LFS
try:
    print("Initializing git-lfs...")
    run_command("git lfs install")  # Initialize git-lfs
    print("Cloning StyleTTS2-LibriTTS from Hugging Face...")
    run_command("git lfs clone https://huggingface.co/yl4579/StyleTTS2-LibriTTS")
except Exception as e:
    print(f"Error while cloning model from Hugging Face: {e}")

# Move the downloaded models to the desired directory
try:
    print("Moving Models directory...")
    shutil.move("StyleTTS2-LibriTTS/Models", "Models")
except Exception as e:
    print(f"Error while moving Models directory: {e}")


Cloning StyleTTS2 repository...
Initializing git-lfs...
Cloning StyleTTS2-LibriTTS from Hugging Face...
Moving Models directory...


In [1]:
import os
import shutil

# Change directory to 'StyleTTS2'
try:
    os.chdir("StyleTTS2")
    print(f"Changed directory to: {os.getcwd()}")
except FileNotFoundError:
    print("Directory 'StyleTTS2' not found.")

Changed directory to: /home/youshanzhang/sahil/projects/sem1/voice_cloning/clone_2/StyleTTS2


In [2]:
os.getcwd()

'/home/youshanzhang/sahil/projects/sem1/voice_cloning/clone_2/StyleTTS2'

In [10]:
# import gdown
# import zipfile
# import os

# # Download the file from Google Drive
# try:
#     file_id = "1vqz26D3yn7OXS2vbfYxfSnpLS6m6tOFP"
#     output = "Data.zip"
#     gdown.download(f"https://drive.google.com/uc?id={file_id}", output, quiet=False)
#     print(f"Downloaded file: {output}")
# except Exception as e:
#     print(f"Error during download: {e}")

# # Unzip the downloaded file
# try:
#     with zipfile.ZipFile(output, 'r') as zip_ref:
#         zip_ref.extractall()  # Extracts to the current directory
#     print(f"Extracted {output} successfully.")
# except FileNotFoundError:
#     print(f"{output} not found.")
# except zipfile.BadZipFile:
#     print(f"{output} is not a valid zip file.")
# except Exception as e:
#     print(f"Error while unzipping: {e}")


In [3]:
config_path = "Configs/config_ft.yml"

import yaml
config = yaml.safe_load(open(config_path))

In [4]:
config['data_params']['root_path'] = "Data/wavs"

config['batch_size'] = 2 # not enough RAM
config['max_len'] = 100 # not enough RAM
config['loss_params']['joint_epoch'] = 110 # we do not do SLM adversarial training due to not enough RAM

with open(config_path, 'w') as outfile:
  yaml.dump(config, outfile, default_flow_style=True)

In [14]:
!python train_finetune.py --config_path ./Configs/config_ft.yml

Some weights of the model checkpoint at microsoft/wavlm-base-plus were not used when initializing WavLMModel: ['encoder.pos_conv_embed.conv.weight_g', 'encoder.pos_conv_embed.conv.weight_v']
- This IS expected if you are initializing WavLMModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing WavLMModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of WavLMModel were not initialized from the model checkpoint at microsoft/wavlm-base-plus and are newly initialized: ['encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'encoder.pos_conv_embed.conv.parametrizations.weight.original1']
You should probably TRAIN this model on a down-stream task to be able to use it for predictio

### Test the model quality

Note that this mainly serves as a proof of concept due to RAM limitation of free Colab instances. A lot of settings are suboptimal. In the future when DDP works for train_second.py, we will also add mixed precision finetuning to save time and RAM. You can also add SLM adversarial training run if you have paid Colab services (such as A100 with 40G of RAM).

In [15]:
import nltk
# nltk.download('all')

In [16]:
# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

In [17]:
import torch
torch.manual_seed(0)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

import random
random.seed(0)

import numpy as np
np.random.seed(0)

# load packages
import time
import random
import yaml
from munch import Munch
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F
import torchaudio
import librosa
from nltk.tokenize import word_tokenize

from models import *
from utils import *
from text_utils import TextCleaner
textclenaer = TextCleaner()

%matplotlib inline

to_mel = torchaudio.transforms.MelSpectrogram(
    n_mels=80, n_fft=2048, win_length=1200, hop_length=300)
mean, std = -4, 4

def length_to_mask(lengths):
    mask = torch.arange(lengths.max()).unsqueeze(0).expand(lengths.shape[0], -1).type_as(lengths)
    mask = torch.gt(mask+1, lengths.unsqueeze(1))
    return mask

def preprocess(wave):
    wave_tensor = torch.from_numpy(wave).float()
    mel_tensor = to_mel(wave_tensor)
    mel_tensor = (torch.log(1e-5 + mel_tensor.unsqueeze(0)) - mean) / std
    return mel_tensor

def compute_style(path):
    wave, sr = librosa.load(path, sr=24000)
    audio, index = librosa.effects.trim(wave, top_db=30)
    if sr != 24000:
        audio = librosa.resample(audio, sr, 24000)
    mel_tensor = preprocess(audio).to(device)

    with torch.no_grad():
        ref_s = model.style_encoder(mel_tensor.unsqueeze(1))
        ref_p = model.predictor_encoder(mel_tensor.unsqueeze(1))

    return torch.cat([ref_s, ref_p], dim=1)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# load phonemizer
import phonemizer
global_phonemizer = phonemizer.backend.EspeakBackend(language='en-us', preserve_punctuation=True,  with_stress=True)

config = yaml.safe_load(open("Models/LJSpeech/config_ft.yml"))

# load pretrained ASR model
ASR_config = config.get('ASR_config', False)
ASR_path = config.get('ASR_path', False)
text_aligner = load_ASR_models(ASR_path, ASR_config)

# load pretrained F0 model
F0_path = config.get('F0_path', False)
pitch_extractor = load_F0_models(F0_path)

# load BERT model
from Utils.PLBERT.util import load_plbert
BERT_path = config.get('PLBERT_dir', False)
plbert = load_plbert(BERT_path)

model_params = recursive_munch(config['model_params'])
model = build_model(model_params, text_aligner, pitch_extractor, plbert)
_ = [model[key].eval() for key in model]
_ = [model[key].to(device) for key in model]

177


/home/youshanzhang/software/anaconda3/envs/genai/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")
/home/youshanzhang/software/anaconda3/envs/genai/lib/python3.12/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


In [18]:
files = [f for f in os.listdir("Models/LJSpeech/") if f.endswith('.pth')]
sorted_files = sorted(files, key=lambda x: int(x.split('_')[-1].split('.')[0]))

In [19]:
params_whole = torch.load("Models/LJSpeech/" + sorted_files[-1], map_location='cpu')
params = params_whole['net']

In [20]:
for key in model:
    if key in params:
        print('%s loaded' % key)
        try:
            model[key].load_state_dict(params[key])
        except:
            from collections import OrderedDict
            state_dict = params[key]
            new_state_dict = OrderedDict()
            for k, v in state_dict.items():
                name = k[7:] # remove `module.`
                new_state_dict[name] = v
            # load params
            model[key].load_state_dict(new_state_dict, strict=False)
#             except:
#                 _load(params[key], model[key])
_ = [model[key].eval() for key in model]

bert loaded
bert_encoder loaded
predictor loaded
decoder loaded
text_encoder loaded
predictor_encoder loaded
style_encoder loaded
diffusion loaded
text_aligner loaded
pitch_extractor loaded
mpd loaded
msd loaded
wd loaded


In [21]:
from Modules.diffusion.sampler import DiffusionSampler, ADPM2Sampler, KarrasSchedule

In [22]:
sampler = DiffusionSampler(
    model.diffusion.diffusion,
    sampler=ADPM2Sampler(),
    sigma_schedule=KarrasSchedule(sigma_min=0.0001, sigma_max=3.0, rho=9.0), # empirical parameters
    clamp=False
)

In [23]:
def inference(text, ref_s, alpha = 0.3, beta = 0.7, diffusion_steps=5, embedding_scale=1):
    text = text.strip()
    ps = global_phonemizer.phonemize([text])
    ps = word_tokenize(ps[0])
    ps = ' '.join(ps)
    tokens = textclenaer(ps)
    tokens.insert(0, 0)
    tokens = torch.LongTensor(tokens).to(device).unsqueeze(0)

    with torch.no_grad():
        input_lengths = torch.LongTensor([tokens.shape[-1]]).to(device)
        text_mask = length_to_mask(input_lengths).to(device)

        t_en = model.text_encoder(tokens, input_lengths, text_mask)
        bert_dur = model.bert(tokens, attention_mask=(~text_mask).int())
        d_en = model.bert_encoder(bert_dur).transpose(-1, -2)

        s_pred = sampler(noise = torch.randn((1, 256)).unsqueeze(1).to(device),
                                          embedding=bert_dur,
                                          embedding_scale=embedding_scale,
                                            features=ref_s, # reference from the same speaker as the embedding
                                             num_steps=diffusion_steps).squeeze(1)


        s = s_pred[:, 128:]
        ref = s_pred[:, :128]

        ref = alpha * ref + (1 - alpha)  * ref_s[:, :128]
        s = beta * s + (1 - beta)  * ref_s[:, 128:]

        d = model.predictor.text_encoder(d_en,
                                         s, input_lengths, text_mask)

        x, _ = model.predictor.lstm(d)
        duration = model.predictor.duration_proj(x)

        duration = torch.sigmoid(duration).sum(axis=-1)
        pred_dur = torch.round(duration.squeeze()).clamp(min=1)

        pred_aln_trg = torch.zeros(input_lengths, int(pred_dur.sum().data))
        c_frame = 0
        for i in range(pred_aln_trg.size(0)):
            pred_aln_trg[i, c_frame:c_frame + int(pred_dur[i].data)] = 1
            c_frame += int(pred_dur[i].data)

        # encode prosody
        en = (d.transpose(-1, -2) @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(en)
            asr_new[:, :, 0] = en[:, :, 0]
            asr_new[:, :, 1:] = en[:, :, 0:-1]
            en = asr_new

        F0_pred, N_pred = model.predictor.F0Ntrain(en, s)

        asr = (t_en @ pred_aln_trg.unsqueeze(0).to(device))
        if model_params.decoder.type == "hifigan":
            asr_new = torch.zeros_like(asr)
            asr_new[:, :, 0] = asr[:, :, 0]
            asr_new[:, :, 1:] = asr[:, :, 0:-1]
            asr = asr_new

        out = model.decoder(asr,
                                F0_pred, N_pred, ref.squeeze().unsqueeze(0))


    return out.squeeze().cpu().numpy()[..., :-50] # weird pulse at the end of the model, need to be fixed later

In [24]:
text = '''Maltby and Company would issue warrants on them deliverable to the importer, and the goods were then passed to be stored in neighboring warehouses.
'''

In [25]:
# get a random reference in the training set, note that it doesn't matter which one you use
path = "Data/wavs/clip_slide3_0010-0020.wav"
# this style vector ref_s can be saved as a parameter together with the model weights
ref_s = compute_style(path)

In [26]:
# # get a random reference in the training set, note that it doesn't matter which one you use
# path = "Data/wavs/LJ001-0110.wav"
# # this style vector ref_s can be saved as a parameter together with the model weights
# ref_s = compute_style(path)

In [27]:
start = time.time()
wav = inference(text, ref_s, alpha=0.9, beta=0.9, diffusion_steps=10, embedding_scale=1)
rtf = (time.time() - start) / (len(wav) / 24000)
print(f"RTF = {rtf:5f}")
import IPython.display as ipd
display(ipd.Audio(wav, rate=24000, normalize=False))

RTF = 0.063336


---